# Goal: obtain structural parameters from simulated data 

In [1]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
tolerance = 0.1

#structural parameters:  UNKNOWN - TO BE ESTIMATED
sigma_guess = [0.2, 0.9] 
parameter_guess = [4,5] 

market_data = pd.read_csv('simulated_data.csv')

In [5]:
# # Main GMM code:
def objective_function(sigma_guess, input_dataset, parameter_guess, tolerance): #return loss value which we want to minimize 
    delta_array = f.mean_utility(input_dataset, sigma_guess, parameter_guess, tolerance)
    input_dataset['mean_utility'] = delta_array
    alpha_hat, beta_hat = f.estimate_parameter(input_dataset)
    # calculating residual (error terms)
    x = input_dataset['x'].values
    p = input_dataset['price'].values
    residual = delta_array - (beta_hat * x + alpha_hat * p)
    Z = input_dataset[['instrument','price']].values
    g = Z.T @ residual
    return g.T @ g #loss function of the residuals 

sigma_guess = [0.2, 0.9] 
parameter_guess = [4,5] 

result = minimize(
    objective_function, 
    x0=sigma_guess, 
    args=(market_data, parameter_guess, tolerance), 
    method= 'Powell', 
    options= {'disp': True}
)

print("found!", result)
sigma_hat = result.x
print("Estimated sigma:", sigma_hat) #true values are [0.7,0.3]
print("Estimated parameter:", f.estimate_parameter(market_data))

#using found estimated sigma result to find alpha and beta
market_data['mean_utility'] = f.mean_utility(market_data, sigma_hat, parameter_guess, tolerance)
alpha_beta = f.estimate_parameter(market_data)
print("Estimated parameter newly found to be:", alpha_beta) #true values are [3,13]

Optimization terminated successfully.
         Current function value: 201.681262
         Iterations: 3
         Function evaluations: 84
found!  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 201.681262404
       x: [ 1.359e+00  1.863e+00]
     nit: 3
   direc: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
    nfev: 84
Estimated sigma: [ 1.359  1.863]
Estimated parameter: (-0.083078923246648451, 6.6835129655859422)
Estimated parameter newly found to be: (-0.069733230117986267, 6.7852664291757137)


In [ ]:
# Extension: Two-step GMM
# Step 1: GMM with identity matrix
def first_stage_loss(sigma_guess, dataset, param_guess):
    delta_array = f.mean_utility(dataset, sigma_guess, param_guess, tolerance)
    dataset['mean_utility'] = delta_array
    alpha_hat, beta_hat = f.estimate_parameter(dataset)
    residuals = delta_array - (beta_hat * dataset['x'].values + alpha_hat * dataset['price'].values)
    Z = dataset[['instrument', 'price']].values
    g = Z.T @ residuals
    return g.T @ g

# Step 2: Optimal weighting matrix and second GMM pass
def second_stage_loss(sigma_guess, dataset, param_guess, W):
    delta_array = f.mean_utility(dataset, sigma_guess, param_guess, tolerance)
    dataset['mean_utility'] = delta_array
    alpha_hat, beta_hat = f.estimate_parameter(dataset)
    residuals = delta_array - (beta_hat * dataset['x'].values + alpha_hat * dataset['price'].values)
    Z = dataset[['instrument', 'price']].values
    g = Z.T @ residuals
    return g.T @ W @ g

# Run 2-step GMM
def two_step_gmm(dataset, tolerance, sigma_init, param_guess):
    # First step
    result1 = minimize(first_stage_loss, x0=sigma_init, args=(dataset, param_guess), method='Nelder-Mead')
    sigma1 = result1.x
    delta1 = f.mean_utility(dataset, sigma1, param_guess, tolerance)
    dataset['mean_utility'] = delta1
    alpha1, beta1 = f.estimate_parameter(dataset)

    # Optimal W
    residual1 = delta1 - (beta1 * dataset['x'].values + alpha1 * dataset['price'].values)
    Z = dataset[['instrument', 'price']].values
    W_opt = inv(Z.T @ Z)

    # Second step
    result2 = minimize(second_stage_loss, x0=sigma1, args=(dataset, param_guess, W_opt), method='Nelder-Mead')
    sigma2 = result2.x
    delta2 = f.mean_utility(dataset, sigma2, param_guess, tolerance)
    dataset['mean_utility'] = delta2
    alpha2, beta2 = f.estimate_parameter(dataset)

    return {
        'step1': {'sigma': sigma1, 'alpha': alpha1, 'beta': beta1},
        'step2': {'sigma': sigma2, 'alpha': alpha2, 'beta': beta2}
    }

result = two_step_gmm(market_data, tolerance, sigma_init=[0.3, 0.4], param_guess=[3, 13])
print("Step 1 Estimates:", result['step1'])
print("Step 2 Estimates:", result['step2'])



KeyboardInterrupt: 